In [7]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages.human import HumanMessage

from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.lats.node import Node
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,
)
from agential.cog.lats.strategies.qa import (
    LATSAmbigNQStrategy,
    LATSFEVERStrategy,
    LATSHotQAStrategy,
    LATSTriviaQAStrategy,
    parse_qa_action,
    parse_qa_value,
)
from agential.cog.react.output import ReActOutput
from agential.utils.docstore import DocstoreExplorer


llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1.0,
    max_tokens=100,
    n=1,
    stop="."
)

/opt/anaconda3/envs/agential/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(


In [8]:
"""Test the simulate_node method."""
qa_strategy = LATSHotQAStrategy(llm=llm)

# Create a mock Node
root_node = Node(depth=0, is_terminal=False)

# Mock input parameters
question = "What is the capital of France?"
key = "Paris"
examples = HOTPOTQA_FEWSHOT_EXAMPLES_REACT
reflect_examples = HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT
value_examples = HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE
prompt = LATS_INSTRUCTION_HOTPOTQA
reflect_prompt = LATS_REFLECT_INSTRUCTION_HOTPOTQA
value_prompt = LATS_VALUE_INSTRUCTION_HOTPOTQA
additional_keys = {}
reflect_additional_keys = {}
value_additional_keys = {}

# Call the simulate_node function
reward, final_node, all_children_nodes, all_values = qa_strategy.simulate_node(
	node=root_node,
	question=question,
	key=key,
	examples=examples,
	reflect_examples=reflect_examples,
	value_examples=value_examples,
	prompt=prompt,
	reflect_prompt=reflect_prompt,
	value_prompt=value_prompt,
	additional_keys=additional_keys,
	reflect_additional_keys=reflect_additional_keys,
	value_additional_keys=value_additional_keys,
)

